In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-03-16"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
16649,2024-03-16,Espanha Acb,16:45,Manresa,Gran Canaria,1.78,2.04,166.5,1.86,1.92,-2.5,1.98,2.30,8Ay8bCC2,0.561798,0.490196,0.537634,0.520833,0.051994,0.6,0.4,NaN,NaN,167.730,46.412515,0.276710,174.526,89.735661,0.514168,167.398,172.732,140.27,82.36,0.0,0.0,0.0,0.0,0.096255,0.022448,0.105736,-1.93,-0.386,-2.020725,0.000000,0.0,0.000000,0.93,0.186,5.591398,0.000000,0.0,0.000000
16650,2024-03-16,Espanha Acb,16:45,MoraBanc Andorra,Tenerife,2.22,1.68,162.5,1.81,1.95,1.5,2.03,1.53,n1zCcWR8,0.450450,0.595238,0.552486,0.512821,0.045689,0.6,0.8,NaN,NaN,254.212,184.929897,0.727463,153.534,33.988416,0.221374,155.014,208.890,120.90,178.08,0.0,0.0,0.0,0.0,0.195814,0.052657,0.198626,-0.10,-0.020,-61.000000,0.000000,0.0,0.000000,2.63,0.526,1.292776,0.000000,0.0,0.000000
16651,2024-03-16,Eua Nba,23:00,Sacramento Kings,New York Knicks,1.66,2.04,218.5,1.81,2.01,-2.5,1.95,2.33,8zmqjbqa,0.602410,0.490196,0.552486,0.497512,0.092606,0.6,0.2,NaN,NaN,212.480,59.856848,0.281706,217.994,31.217749,0.143205,185.760,201.590,244.80,257.87,0.0,0.0,0.0,0.0,0.145244,0.074043,0.125561,1.20,0.240,2.750000,0.705458,0.6,-0.105458,0.76,0.152,6.842105,0.562841,0.7,0.137159
16652,2024-03-16,Alemanha Bbl,14:30,Heidelberg,Rostock,1.74,2.10,180.5,1.83,1.95,-2.5,1.97,2.32,QPYJKIW0,0.574713,0.476190,0.546448,0.512821,0.050903,0.8,0.8,NaN,NaN,339.440,193.369840,0.569673,241.608,68.262715,0.282535,215.830,318.464,170.24,209.10,0.0,0.0,0.0,0.0,0.132583,0.044896,0.115379,-5.00,-1.000,-0.740000,0.000000,0.0,0.000000,1.40,0.280,3.928571,0.000000,0.0,0.000000
16653,2024-03-16,Argentina Liga A,11:30,San Lorenzo,Gimnasia,1.77,1.95,155.5,1.87,1.79,-1.5,1.91,2.11,4xmuT16J,0.564972,0.512821,0.534759,0.558659,0.077792,0.4,0.6,NaN,NaN,207.522,105.841873,0.510027,122.886,30.625287,0.249217,135.974,138.244,246.05,110.88,0.0,0.0,0.0,0.0,0.068430,0.030912,0.070359,-1.04,-0.208,-3.701923,0.632774,0.7,0.067226,-5.00,-1.000,-0.950000,0.000000,0.0,0.000000
16654,2024-03-16,Austrália Liga Feminina Wnbl,23:30,Southside F,Perth Lynx F,2.09,1.71,174.5,1.86,1.88,1.5,1.92,1.55,lzttCAzb,0.478469,0.584795,0.537634,0.531915,0.063264,0.4,0.6,NaN,NaN,164.380,64.257482,0.390908,173.088,51.034209,0.294845,114.050,182.398,154.38,247.10,0.0,0.0,0.0,0.0,0.141421,0.007563,0.150795,0.70,0.140,7.785714,0.673629,0.7,0.026371,-5.00,-1.000,-0.710000,0.000000,0.0,0.000000
16655,2024-03-16,Bósnia E Herzegovina Prvenstvo Bih,15:00,Prijedor,Mrkonjic Grad,2.01,1.73,162.5,1.80,1.86,1.5,1.85,1.61,Asz5LtCH,0.497512,0.578035,0.555556,0.537634,0.075547,0.6,0.8,NaN,NaN,295.966,255.288328,0.862560,307.290,237.488557,0.772848,295.686,316.846,181.80,296.56,0.0,0.0,0.0,0.0,0.105877,0.023184,0.098096,-2.68,-0.536,-1.884328,0.000000,0.0,0.000000,1.67,0.334,2.185629,0.000000,0.0,0.000000
16656,2024-03-16,Chile Lnb,20:00,Las Animas,Valdivia,1.99,1.74,165.5,1.87,1.79,1.5,1.85,1.61,zXkxWpZI,0.502513,0.574713,0.534759,0.558659,0.077225,0.0,0.0,NaN,NaN,141.972,33.884213,0.238668,110.206,29.247601,0.265390,0.000,0.000,144.20,144.00,0.0,0.0,0.0,0.0,0.094786,0.030912,0.098096,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
16657,2024-03-16,Chile Lnb,20:15,Puerto Montt,Catolica,1.98,1.75,150.5,1.84,1.86,-1.5,1.85,1.85,n92qvrRu,0.505051,0.571429,0.543478,0.537634,0.076479,0.6,1.0,NaN,NaN,324.626,266.845095,0.822008,308.388,328.444250,1.065036,276.152,542.508,146.16,283.08,0.0,0.0,0.0,0.0,0.087204,0.007644,0.000000,-1.29,-0.258,-3.798450,0.0000

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,16:45,Espanha Acb,Manresa,Gran Canaria,166.5,1.86,1.0000,Over
1,23:00,Eua Nba,Sacramento Kings,New York Knicks,218.5,1.81,1.0000,Over
2,14:30,Alemanha Bbl,Heidelberg,Rostock,180.5,1.83,1.0000,Over
3,11:30,Argentina Liga A,San Lorenzo,Gimnasia,155.5,1.87,1.0000,Over
4,08:35,China Cba,Shanxi Zhongyu,Nanjing Tongxi,225.5,1.87,1.0000,Over
5,15:00,Eslovênia Liga Nova Kbm,Tajfun Sentjur,KK Triglav Kranj,154.5,1.80,0.6914,Over
6,15:00,Espanha Leb Ouro,Alicante,Estudiantes,156.5,1.80,1.0000,Over
7,14:30,Espanha Liga Feminina,Gernika Bizkaia F,Jairis F,136.5,1.85,1.0000,Over
8,00:30,Eua Ncaa,Tarleton,Texas Arlington,145.5,1.91,1.0000,Over
9,00:30,Eua Ncaa,UC Davis,Hawaii,134.5,1.80,1.0000,Over
